In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline
import pandas as pd 
import scipy.stats as stats
import seaborn as sns


In [2]:
# get experimental directories
data_directory = '/projects/bdata2/galen/causal_backups/'

exp_dirs = [name for name in os.listdir(data_directory) if ('_60' in name and name[:3] == 'exp' and os.path.isdir(data_directory+name))] # or ('_40' in name and os.path.isdir(name)) or ('_60' in name and os.path.isdir(name))]
all_exp_dirs = [name for name in os.listdir(data_directory) if (name[:3] == 'exp' and os.path.isdir(data_directory+name))] # or ('_40' in name and os.path.isdir(name)) or ('_60' in name and os.path.isdir(name))]

exp_dirs.remove('experiment_0_0_0_1_0_60_backup')

# get experiment features
get_features = lambda d: [int(v) for v in d.split('_')[1:6]]

# get size
get_size = lambda d: d.split('_')[6]

print(exp_dirs)
print(all_exp_dirs)


['experiment_1_0_0_0_0_60_nuser2000', 'experiment_0_0_0_0_1_60', 'experiment_0_0_0_0_0_60', 'experiment_1_0_0_0_0_60_nuser4000', 'experiment_0_0_0_1_0_60', 'experiment_0_1_0_0_0_60', 'experiment_2_0_0_0_0_60', 'experiment_3_0_0_0_0_60', 'experiment_4_0_0_0_0_60', 'experiment_1_0_0_0_0_60', 'experiment_4_0_0_0_0_60_backup', 'experiment_0_0_1_0_0_60', 'experiment_0_0_0_2_0_60']
['experiment 3', 'experiment_6', 'experiment_3_0_0_0_0_30', 'experiment_1_0_0_0_0_20', 'experiment_realdata_galen', 'experiment_0_0_0_0_2_xsmall', 'experiment_1_0_0_0_0_60_nuser2000', 'experiment_1_xsmall', 'experiment_8_min_pw', 'experiment_0_0_1_0_0_20', 'experiment_0_0_0_0_1_60', 'experiment_backups', 'experiment_0_1_0_0_0_xsmall', 'experiment_8_min', 'experiment_3_xsmall', 'experiment_realdata_suicide_med', 'experiment_0_0_0_0_0_60', 'experiment_6_min', 'experiment_1', 'experiment 4', 'experiment_0_0_0_0_0_xsmall', 'experiment_1_min', 'experiment_1_0_0_0_0_30', 'experiment_realdata_useless_med', 'experiment_0_

In [3]:



axis_0 = {'name': 'Linguistic Complexity',
          'unadjusted':{'test_acc':0.5, 'bias_strat':-0.32, 'bias_ipw':-0.32,'spearman':0.},
          'optimal':{'test_acc':0.9, 'bias_strat':0., 'bias_ipw':0.,'spearman':1., 'mse_ipsw':0.},
          'labels': ['constant','+sick','+isolation', '+death' ],
          
          'experiments':[
              {'file_path': 'experiment_0_0_0_0_0_60',
               'axis_value': 0,#0,
               'Y_sample_classes': [[0.9,0.1],[0.9,0.9]],
               'P_sample_classes': [0.9,0.1],
              'unadjusted':{'test_acc':0.5, 'bias_strat':-0.32, 'bias_ipw':-0.32,'spearman':0.},
              'optimal':{'test_acc':0.9, 'bias_strat':0., 'bias_ipw':0.,'spearman':1.}
              },
              
              {'file_path': 'experiment_1_0_0_0_0_60',
               'axis_value': 1,#1,
               'Y_sample_classes': [[0.9,0.1],[0.9,0.9]],
               'P_sample_classes': [0.9,0.1],
              'unadjusted':{'test_acc':0.5, 'bias_strat':-0.32, 'bias_ipw':-0.32,'spearman':0.},
              'optimal':{'test_acc':0.9, 'bias_strat':0., 'bias_ipw':0.,'spearman':1.}},
              
              {'file_path': 'experiment_2_0_0_0_0_60',
               'axis_value': 2,#2,
               'Y_sample_classes': [[0.9,0.1],[0.9,0.9]],
               'P_sample_classes': [0.9,0.1],
              'unadjusted':{'test_acc':0.5, 'bias_strat':-0.32, 'bias_ipw':-0.32,'spearman':0.},
              'optimal':{'test_acc':0.9, 'bias_strat':0., 'bias_ipw':0.,'spearman':1.}},
    
              {'file_path': 'experiment_3_0_0_0_0_60',
               'axis_value': 3,#3,
               'Y_sample_classes': [[0.9,0.1],[0.9,0.9]],
               'P_sample_classes': [0.9,0.1],
              'unadjusted':{'test_acc':0.5, 'bias_strat':-0.32, 'bias_ipw':-0.32,'spearman':0.},
              'optimal':{'test_acc':0.9, 'bias_strat':0., 'bias_ipw':0.,'spearman':1.}},

              {'file_path': 'experiment_4_0_0_0_0_60_backup',
               'axis_value': 4,
               'Y_sample_classes': [[0.9,0.1],[0.9,0.9]],
               'P_sample_classes': [0.9,0.1],
              'unadjusted':{'test_acc':0.5, 'bias_strat':-0.32, 'bias_ipw':-0.32,'spearman':0.},
              'optimal':{'test_acc':0.9, 'bias_strat':0., 'bias_ipw':0.,'spearman':1.}},
              
          ]}

axis_0['experiments'] = axis_0['experiments'][:4]


In [24]:
exp_name = axis_0['experiments'][0]['file_path']
Y_sample_classes = axis_0['experiments'][0]['Y_sample_classes'] #[[0.9,0.1],[0.9,0.9]]
P_sample_classes = axis_0['experiments'][0]['P_sample_classes']

inds_dict = np.load(data_directory + exp_name + '/inds_dict.npy', allow_pickle=True).item()
_, _, T, Y, classes = np.load(data_directory+exp_name +'/data.npy', allow_pickle=True)
P_true = P_sample_classes[0]*(1-classes) + P_sample_classes[1]*classes

model_name = 'HBERT'
stat_dict = np.load(data_directory+'{}/{}.npy'.format(exp_name, model_name), allow_pickle=True).item()

P_test = stat_dict['P_test']
Tt_test = T[inds_dict['inds_test']]
Y_sample = np.zeros_like( P_true[inds_dict['inds_test']] )
classes_test = classes[inds_dict['inds_test']]
for i in range(len(classes_test)):
    Y_sample[i] = 1.*(random.random() < Y_sample_classes[classes_test[i]][1*(Tt_test[i] == 0)])


ATE_true = 0.5*((Y_sample_classes[0][0] - 
                 Y_sample_classes[0][1]) + 
                (Y_sample_classes[1][0] - 
                Y_sample_classes[1][1]))

In [27]:
np.unique(P_test)

array([0.03198493, 0.03706409, 0.03843177, ..., 0.86624557, 0.86624569,
       0.86624575])

In [20]:
def get_effect_match(P,Z,Y, replacement=True, caliper=None, warning_tol=.1):
    '''
    full matching = each (or some?) control matched to one or more treated subjects (rosenbaum, 179) *without* replacement
    
    our implementation is from https://onlinelibrary.wiley.com/doi/abs/10.3982/ECTA11293, page 784
    
    caliper?
    We enforce a propensity score caliper equal to 0.1 standard
    deviations of the estimated distribution, which discards any treated units for whom the
    nearest control unit is not within a suitable distance.
    from Mozer 2018: https://arxiv.org/abs/1801.00644, page 31
     -> does this mean the distribution of all (both treated and untreated) propensity scores?
     
     
    then again, rosenbaum (p. 169) recomends starting with .2
    
    vary caliber in terms of the standard deviation of the propensity scores, eg. .2 for 20% of a std away
    '''
    if not replacement:
        assert(False) # not implemented yet (but it should be)
        
    P_control = P[Z == 0]
    Y_control = Y[Z == 0]
    
    P_treat = P[Z == 1]
    Y_treat = Y[Z == 1]
    
    # if we match all the treated to the closest untreated with replacement, we have ATE on Treated (ATT)
    # if we match all the untreated to the closest treated with replacement, we have ATE on Untreated (ATU?)
    # I have no idea if this works, but let's try doing both to compute ATE
    
    diffs = [] # difference in outcome (treated - control)
    dists = [] # differenence in propensity score (for caliper application)
    
    # todo!!! also implement this without replacement
    for treat_index, p in enumerate(P_treat):        
        # get match for p (closest point in control by propensity)
        control_index = np.argmin( np.abs(P_control - p) )
        diffs.append( Y_treat[treat_index] - Y_control[control_index] )
        dists.append( np.abs( P_control[control_index] - p ) )
        
    for control_index, p in enumerate(P_control):
        # get match for p (closest point in treat by propensity)
        treat_index = np.argmin( np.abs(P_treat - p) )
        diffs.append( Y_treat[treat_index] - Y_control[control_index] )
        dists.append( np.abs( P_treat[treat_index] - p ) )
        
    #print(f'Average distances with caliper {caliper} are {np.average(dists):8.5f}')
    #print(f'Propensity score STD with caliper {caliper} are {np.std(P):8.5f}')
    #print(f'Propensity score STD on treated with caliper {caliper} are {np.std(P_treat):8.5f}')
    #print(f'Propensity score STD with on untreated caliper {caliper} are {np.std(P_control):8.5f}')
    
    
    
    
    # CALIPER SECTION IS AGNOSTIC TO REPLACEMENT IMPLEMENTATION
    # if using caliper, discard invalid matches
    if caliper is not None:        
        Valid = dists < (caliper * P.std()) # caliper is fraction of standard deviation from the distribution
        
        #print(f'discarded {np.sum(-1*Valid+1)} by caliper ({100*(1-(np.sum(Valid)/Valid.size)):4.2f}%)')
        if (1. - 1.*np.sum(Valid)/Valid.size) > warning_tol:
            print('WARNING: discarding {} of examples, exceeds tolerance of {}'.format((1. - 1.*np.sum(Valid)/Valid.size), warning_tol))
        
        diffs = np.array(diffs)[Valid]
        dists = np.array(dists)[Valid]
        
    #print(f'Average distances after caliper are {np.average(dists):8.5f}')
        
    #print(f'ATE is {np.average(diffs):8.5f}, bias is {np.average(diffs)-ATE_true:8.5f}\n')
    return np.average(diffs)

In [21]:
#get_effect_match(P_test, Tt_test, Y_sample, caliper=None)
get_effect_match(P_test, Tt_test, Y_sample, caliper=.2)

0.3615

In [22]:
bootstrapped_estimates = []
for _ in range(100):
    bootsrapped_indices = np.random.choice(len(P_test), size=len(P_test), replace=True)
    
    bootstrapped_estimates.append(  get_effect_match(P_test[bootsrapped_indices], Tt_test[bootsrapped_indices], Y_sample[bootsrapped_indices], caliper=.2) - ATE_true   )

In [23]:
np.array(bootstrapped_estimates).std()

0.01595845602650833

In [36]:
diffs = np.array((.1, .1, .1, .9, .1, .9, 8))

In [41]:
diffs==min(diffs)

array([ True,  True,  True, False,  True, False, False])

In [45]:
(diffs==min(diffs)).nonzero()

(array([0, 1, 2, 4]),)

In [47]:
np.random.choice((diffs==min(diffs)).nonzero()[0] ) 

4